In [26]:
import requests

# API 키와 기본 URL 설정
api_key = '18a4025554d765b6434de68f0e4dd1a1a0ffb5f8'

In [27]:
def strToInt(s):
    return int(s.replace(",",""))

### 회사별 고유번호

In [28]:
import zipfile
import os, csv
import xml.etree.ElementTree as ET

corpCode_url = 'https://opendart.fss.or.kr/api/corpCode.xml'

# 요청 파라미터 설정
params = {
    'crtfc_key': api_key
}

# API 호출
response = requests.get(corpCode_url, params=params)

# 응답 데이터 처리
if response.status_code == 200:
    with open('corpCode.zip', 'wb') as f:
        f.write(response.content)
    
    # zip 파일 압축 해제
    with zipfile.ZipFile('corpCode.zip', 'r') as zip_ref:
        zip_ref.extractall()

    # XML 파일 파싱
    tree = ET.parse('CORPCODE.xml')
    root = tree.getroot()

    # CSV 파일 작성
    with open('corp_info.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['stock_code', 'corp_code', 'corp_name']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for company in root.findall('list'):
            corp_code = company.find('corp_code').text
            corp_name = company.find('corp_name').text
            stock_code = company.find('stock_code').text
            if stock_code and len(stock_code) >= 2:
                writer.writerow({'stock_code': str(stock_code), 'corp_code': str(corp_code), 'corp_name': corp_name})
    
    # 압축 파일 및 XML 파일 삭제 (선택 사항)
    os.remove('corpCode.zip')
    os.remove('CORPCODE.xml')
else:
    print(f'Error: {response.status_code}')

### 정기보고서 재무정보 - 단일회사 주요계정
- 여기서 부채비율, 유동비율을 계산할 수 있을 것 같다.

In [21]:
import json
account_url = 'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json'

# 요청 파라미터 설정
params = {
    'crtfc_key': api_key,
    'corp_code': '00126380',  # 예시: 삼성전자
    'bsns_year': '2023',      # 사업 연도
    'reprt_code': '11011'     # 반기 보고서
}

# API 호출
response = requests.get(account_url, params=params)

# 응답 데이터 처리
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=4, ensure_ascii=False))
else:
    print(f'Error: {response.status_code}')


{
    "status": "000",
    "message": "정상",
    "list": [
        {
            "rcept_no": "20240312000736",
            "reprt_code": "11011",
            "bsns_year": "2023",
            "corp_code": "00126380",
            "stock_code": "005930",
            "fs_div": "CFS",
            "fs_nm": "연결재무제표",
            "sj_div": "BS",
            "sj_nm": "재무상태표",
            "account_nm": "유동자산",
            "thstrm_nm": "제 55 기",
            "thstrm_dt": "2023.12.31 현재",
            "thstrm_amount": "195,936,557,000,000",
            "frmtrm_nm": "제 54 기",
            "frmtrm_dt": "2022.12.31 현재",
            "frmtrm_amount": "218,470,581,000,000",
            "bfefrmtrm_nm": "제 53 기",
            "bfefrmtrm_dt": "2021.12.31 현재",
            "bfefrmtrm_amount": "218,163,185,000,000",
            "ord": "1",
            "currency": "KRW"
        },
        {
            "rcept_no": "20240312000736",
            "reprt_code": "11011",
            "bsns_year": "2023",
            "cor

### 임원 현황

In [22]:
url = 'https://opendart.fss.or.kr/api/exctvSttus.json'

# 요청 파라미터 설정
params = {
    'crtfc_key': api_key,
    'corp_code': '00126380',  # 예시: 삼성전자
    'bsns_year': '2023',      # 사업 연도
    'reprt_code': '11011'     # 반기 보고서
}

# API 호출
response = requests.get(url, params=params)

# 응답 데이터 처리
if response.status_code == 200:
    data = response.json()
    # JSON 데이터를 보기 좋게 출력
    print(json.dumps(data, indent=4, ensure_ascii=False))

    # JSON 파일로 저장
    with open('exctvSttus.json', 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)
else:
    print(f'Error: {response.status_code}')


{
    "status": "000",
    "message": "정상",
    "list": [
        {
            "rcept_no": "20240312000736",
            "corp_cls": "Y",
            "corp_code": "00126380",
            "corp_name": "삼성전자",
            "nm": "한종희",
            "sexdstn": "남",
            "birth_ym": "1962년 03월",
            "ofcps": "부회장",
            "rgist_exctv_at": "사내이사",
            "fte_at": "상근",
            "chrg_job": "ㆍ대표이사   (DX 부문 경영전반 총괄)",
            "main_career": "ㆍ인하대 전자공학 학사ㆍ삼성전자 DX부문장",
            "mxmm_shrholdr_relate": "계열회사\n임원",
            "hffc_pd": "46개월",
            "tenure_end_on": "2026년 03월 17일"
        },
        {
            "rcept_no": "20240312000736",
            "corp_cls": "Y",
            "corp_code": "00126380",
            "corp_name": "삼성전자",
            "nm": "허은녕",
            "sexdstn": "남",
            "birth_ym": "1964년 08월",
            "ofcps": "이사",
            "rgist_exctv_at": "사외이사",
            "fte_at": "비상근",
            "chrg_job": "ㆍ사외이사후보

### 정기보고서 재무정보 - 단일회사 주요 재무지표

- 재무제표 : 단일 기업에 대한 재무제표
- 연결재무제표 : 모회사와 그 자회사를 포함한 기업 그룹의 재무상태와 성과를 반영

-> 따라서 연결재무제표를 사용하는 것이 더 낫다

#### 주당순이익 (Earnings Per Share, EPS)
- 주당순이익은 회사의 순이익을 총 발행 주식 수로 나누어 계산합니다.
- 필요한 항목
- 순이익 (Net Income): 손익계산서(Income Statement)에 나와 있습니다.
- 총 발행 주식 수 (Total Number of Outstanding Shares): 주식 발행 관련 보고서나 주석, 또는 회사의 연차 보고서에서 확인할 수 있습니다.
- 계산식
- EPS = 순이익 / 총 발행 주식 수

#### 주당장부가치 (Book Value Per Share, BVPS)
- 주당장부가치는 회사의 장부가치를 총 발행 주식 수로 나누어 계산합니다. 장부가치는 총 자산에서 총 부채를 뺀 값입니다.
- 필요한 항목
- 총 자산 (Total Assets): 대차대조표(Balance Sheet)에 나와 있습니다.
- 총 부채 (Total Liabilities): 대차대조표(Balance Sheet)에 나와 있습니다.
- 총 발행 주식 수 (Total Number of Outstanding Shares): 주식 발행 관련 보고서나 주석, 또는 회사의 연차 보고서에서 확인할 수 있습니다.
- 계산식
- BVPS = (총 자산 - 총 부채) /총 발행 주식 수

In [18]:
import json
fnlttSinglIndx_url = 'https://opendart.fss.or.kr/api/fnlttSinglIndx.json'

# 요청 파라미터 설정
params = {
    'crtfc_key': api_key,
    'corp_code': '00126380',  # 예시: 삼성전자
    'bsns_year': '2023',      # 사업 연도
    'reprt_code': '11011',     # 사업 보고서
    'idx_cl_code':'M210000'
}

# API 호출
response = requests.get(fnlttSinglIndx_url, params=params)

# 응답 데이터 처리
if response.status_code == 200:
    data = response.json()
    # JSON 데이터를 보기 좋게 출력
    print(json.dumps(data, indent=4, ensure_ascii=False))

else:
    print(f'Error: {response.status_code}')

{
    "status": "000",
    "message": "정상",
    "list": [
        {
            "reprt_code": "11011",
            "bsns_year": "2023",
            "corp_code": "00126380",
            "stock_code": "005930",
            "idx_cl_code": "M210000",
            "idx_cl_nm": "수익성지표",
            "idx_code": "M211100",
            "idx_nm": "세전계속사업이익률",
            "idx_val": "5.981"
        },
        {
            "reprt_code": "11011",
            "bsns_year": "2023",
            "corp_code": "00126380",
            "stock_code": "005930",
            "idx_cl_code": "M210000",
            "idx_cl_nm": "수익성지표",
            "idx_code": "M211200",
            "idx_nm": "순이익률",
            "idx_val": "5.981"
        },
        {
            "reprt_code": "11011",
            "bsns_year": "2023",
            "corp_code": "00126380",
            "stock_code": "005930",
            "idx_cl_code": "M210000",
            "idx_cl_nm": "수익성지표",
            "idx_code": "M211250",
            "idx_nm